In [3]:
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader,  random_split

In [4]:
np.random.seed(1337)
random.seed(1337)

In [5]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=100, noise=0.1)
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
generator = torch.Generator().manual_seed(42)
dataset = TensorDataset(X,y)
train_dataset, test_dataset = random_split(dataset, [80, 20],  generator = generator)

: 

In [ ]:
train_features, train_labels = zip(*[(data[0], data[1]) for data in train_dataset])
train_features = torch.stack(train_features)
train_labels = torch.tensor(train_labels)

plt.figure(figsize=(5,5))
plt.scatter(train_features[:,0], train_features[:,1], c=train_labels, s=20, cmap='jet')

: 

In [ ]:
torch.manual_seed = 42
class MLP(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.layers = nn.Sequential(
            nn.Linear(2, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 2),
            nn.ReLU(),
            nn.Linear(2, 1)
        )
    def forward(self, x):
        return self.layers(x)
    
model = MLP()


: 

In [ ]:
def visualise(test=False):
    h = 0.25
    if not test:
        train_features, train_labels = zip(*[(data[0], data[1]) for data in train_dataset])
        X = torch.stack(train_features)
        y = torch.tensor(train_labels)
    else:
        test_features, test_labels = zip(*[(data[0], data[1]) for data in test_dataset])
        X = torch.stack(test_features)
        y = torch.tensor(test_labels)
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                        np.arange(y_min, y_max, h))
    Xmesh = np.c_[xx.ravel(), yy.ravel()]
    inputs = torch.tensor(Xmesh, dtype=torch.float32)
    scores = model(inputs)
    Z = np.array([s.item() > 0 for s in scores])
    Z = Z.reshape(xx.shape)
    fig = plt.figure()
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.8)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.Spectral)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())



: 

In [ ]:

dataloader = DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=True)
loss_function = torch.nn.BCEWithLogitsLoss()
optimiser = torch.optim.Adam(model.parameters(), lr=1e-4)
for i in range(10000):
    total_loss = 0
    for features, labels in dataloader:
        optimiser.zero_grad()
        outputs = model(features).squeeze()
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()
        total_loss += loss.item()
    print(f'Epoch {i}, Avg Loss: {total_loss / len(dataloader)}')



: 

In [ ]:
visualise(test=True)


: 